## Network Link-to-Link attribute Validation Checks

Functions used to identify Link-to-Link pairs and compare values. This ncludes:
- Single Attribute In-Link checks
- Single Attribute In-Link / Out-Link checks
- Multiple Attribute In-Link / Out-Link checks

#### Import Libraries 

In [1]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
# from nxviz import CircosPlot
from shapely.geometry import LineString

#### Load Network data from Shapefiles

In [2]:
def segment_heading(line, first_segment=True, flip=False):
    if flip:
        line = LineString(line.coords[::-1])
    
    segment = line.coords[:2] if first_segment else line.coords[-2:]
    
    from_pt = segment[0]
    to_pt = segment[1]
    
    y_delta = to_pt[1] - from_pt[1]
    x_delta = to_pt[0] - from_pt[0]

    angle_radians = np.arctan2(y_delta, x_delta)
    if angle_radians < 0:
        angle_radians = 2 * np.pi + angle_radians;

    # Deal with the angles being from East Counterclockwise to Northerly bearings
    degrees = 90 - angle_radians * 180 / np.pi
    if degrees < 0:
        degrees = degrees + 360

    return degrees

In [3]:
line_shp = gpd.read_file(os.path.join('..', 'data', 'Chatt_Master.shp')).to_crs(epsg=2274)
node_shp = gpd.read_file(os.path.join('..', 'data', 'Chatt_Master_Node.shp')).to_crs(epsg=2274)

# Heading AB
line_shp['AB_START_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=True)
line_shp['AB_END_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=False)
# Heading BA
line_shp['BA_START_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=True, flip=True)
line_shp['BA_END_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=False, flip=True)

### Build Network Graph

#### Turn Two-Way Links into 2 One-Way Links for Directional Flow.
#### Create Function: *directional_links*
Keep AB Links and produce BA Links, carrying over link attributes based on the direction of flow.

In [4]:
def directional_links(atts, line_shp):
    dirs = {'keep':'AB', 'invert':'BA'}
    dfs = {}
    
    for flow,d in dirs.items():
        links = line_shp[line_shp['{}_LANES'.format(d)] > 0].copy()
        rename_dict = {'{}{}'.format(d, a):a for a in atts}
        links = links.rename(columns=rename_dict)
        
        if flow == 'invert':
            links['A'] = links['TO_ID']
            links['B'] = links['FROM_ID']
            links = links.drop(columns=['FROM_ID', 'TO_ID']).rename(columns={'A': 'FROM_ID', 'B': 'TO_ID'})
        
        dfs['{}_links'.format(d)] = links
    
    links = pd.concat(dfs.values(), ignore_index=True)
    
    drop_dict = ['{}{}'.format(d, a) for a in atts for d in dirs.values()]
    links = links.drop(columns=drop_dict)
    
    return links

#### Generate Directional links with Attributes
####  Call Function: *directional_links*
Use list of Directional Attributes to carry over, e.g. 'AB_LANES' and 'BA_LANES', to '_LANES' 

In [5]:
# List of Directional Attributes
atts = [
    '_LINKDIR',
    '_LINKDTR',
    '_LANES',
    '_PARKING',
    '_TRIMS',
    '_AADT',
    '_CAR_ADT',
    '_SUT_ADT',
    '_MUT_ADT',
    '_BASEVOL',
    '_AFFTIME',
    '_AFFSPD',
    '_UCDELAY',
    '_DLYCAP',
    '_AMCAP',
    '_PMCAP',
    '_BPRA',
    '_BPRB',
    '_START_HEADING',
    '_END_HEADING'
]
# Generate Directional Links
line_shp = directional_links(atts, line_shp)

####  Build the Directional Graph: *g*
#### Call Method: *nx.from_pandas_edgelist*

In [6]:
g = nx.from_pandas_edgelist(
    line_shp,
    source='FROM_ID',
    target='TO_ID',
    edge_attr=True,
    create_using=nx.DiGraph()
)

assert nx.is_directed(g)

### Link to Link validation Functions

#### Create Function: *link_matcher_atts*: 
For each node in Network, match all combinations of In/ Out Links and return both In and Out values

In [7]:
def link_matcher_atts(attributes, graph, edge_id='ID'):
    heading = True if '_HEADING' in attributes else False
    atts = []
    atts.extend([a for a in attributes if a!='_HEADING'])
    
#     if heading: atts.remove('_HEADING')

    node_feed = []

    for node in node_shp['ID']:
        if node not in g:
            print('Node not in network: {}'.format(node))
            continue

        for i_o, i_d, i_data  in g.in_edges(node, data=True):
            for o_o, o_d, o_data in g.out_edges(node, data=True):
                if i_o == o_d and i_d == o_o:
                    # This is just a U-Turn on the "same" link
                    continue
                    
                data = [i_data, o_data]
                values = [node, *[d[edge_id] for d in data], *[d[a] for a in atts for d in data]]
                if heading: values.extend([i_data['_END_HEADING'], o_data['_START_HEADING']])
                node_feed.append(values)
                
    # Generate Column names
    dirs = ['in', 'out']
    cols = ['node', *[ d+'_'+edge_id for d in dirs], *[ d+'_'+a for a in atts for d in dirs]]
    if heading: cols.extend([ d+'__HEADING' for d in dirs])

    return pd.DataFrame(node_feed, columns=cols)

#### Create Function: *attribute_change*: 
Calculate change in Attribute

In [8]:
def attribute_change(network, attributes):
    for att in attributes:
        in_fld = 'in_{}'.format(att)
        out_fld = 'out_{}'.format(att)
        fld_diff = 'diff_{}'.format(att)
#         abs_fld_diff = 'abs_diff_{}'.format(filter_name)
    
        if isinstance(network[in_fld][0], (int, float)):
            network[fld_diff] = network[out_fld] - network[in_fld]
        elif isinstance(network[in_fld][0], str):
            network[fld_diff] = 'SAME' if network[out_fld].str == network[in_fld].str else 'CHANGES'

    return network#.groupby([in_fld, out_fld]).count().reset_index()

#### Create Function: *attribute_filtering*: 
Apply Attributes and Values difference to use as filters

In [9]:
def attribute_filtering(network, att_filters):
    for filter_name, filter_val in att_filters.items():
        in_fld = 'in_{}'.format(filter_name)
        out_fld = 'out_{}'.format(filter_name)
        fld_diff = 'diff_{}'.format(filter_name)
#         abs_fld_diff = 'abs_diff_{}'.format(filter_name)
    
        if isinstance(filter_val, (int, float)):
            network = network[abs(network[fld_diff]) >= filter_val]
        elif isinstance(filter_val, str):
            network = network[(network[in_fld] == filter_val) | (network[out_fld] == filter_val)]

    return network#.groupby([in_fld, out_fld]).count().reset_index()

#### Create Function: *export_geo*: 
Export Geometry to Shapefile

In [10]:
# In/Out Long Table
def export_geo(network, att_name, att_name_add=None):
    fld_diff = 'diff_{}'.format(att_name)
    
    network = network.rename(columns={'out_ID':'ID'})

    # geometries
    geo_cols = ['ID', 'FROM_ID', 'geometry']
    network = pd.merge(line_shp[geo_cols], network, left_on=['ID', 'FROM_ID'], right_on=['ID', 'node'])
    network.drop(columns=['FROM_ID'], inplace=True)
    
    shp_export = os.path.join('..', 'data', 'Network_attribute_change_{}.shp'.format('_'.join([n for n in att_name])))
    network.to_file(shp_export)
#     return network#.sort_values(by=fld_diff, ascending=False), shp_export
    return network, shp_export

#### Create Function: *link_compare*: 
Create function for Link-to-Link attribute comparison

In [11]:
def link_compare(attribute_names, attribute_filters, export_geometry):
    network = link_matcher_atts(attribute_names, graph=g)
    network = attribute_change(network, attribute_names)
    network = attribute_filtering(network,
                                attribute_filters,
                                )
    csv_export = os.path.join('..', 'data', 'Network_attribute_change_table_{}.csv'.format('_'.join([n for n in attribute_names])))
    network.to_csv(csv_export, index=False)
    
    # Print
    print_txt = 'Total network links with change in {}: {:,}'
    print(print_txt.format(', '.join([n for n in attribute_names]),
                           len(network)
                           ))
    print('Table exported to {}'.format(csv_export))
    if export_geometry:
        network, shp_export = export_geo(network, attribute_filters)
        print('Shapefile exported to {}'.format(shp_export))

    return network

#### Create Function: *summary_in*: 
Create function to aggregate In-link by Attribute and summarize by count

In [12]:
def summary_in(network, att, csv_name, filter_vals=''):
    in_att = 'in_{}'.format(att)
    out_att = 'out_{}'.format(att)
    
    if filter_vals is not '':
        network = network[network[out_att].isin(filter_vals)]
        
    csv_export = os.path.join('..', 'data', '{}_{}.csv'.format(csv_name, att))
    network.to_csv(csv_export, index=False)
    
    return network.groupby([in_att])[in_att].count().reset_index(name='out{}'.format(filter_vals))

#### Create Function: *summary_in_out*: 
Create function to aggregate In-link and Out-Link by Attribute and summarize by count

In [13]:
def summary_in_out(network, att, csv_name, filter_in='', filter_out='', return_summary = True, include_same_att=False):
    in_att = 'in_{}'.format(att)
    out_att = 'out_{}'.format(att)
    
    if include_same_att == False:
        network = network[network[in_att]!=network[out_att]]
        
    if filter_in is not '':
        network = network[network[in_att].isin(filter_in)]
        
    if filter_out is not '':
        network = network[network[out_att].isin(filter_out)]
        
    if return_summary == True:
        network = network.groupby([in_att, out_att])[in_att].count().reset_index(name='count')
    
    csv_export = os.path.join('..', 'data', '{}_{}_SUMMARY.csv'.format(csv_name, *attribute_name))
    network.to_csv(csv_export, index=False)

    return network

## Run Network VALIDATION TESTS
Use Heading ('_HEADING') to calculate deviation of In-Link(Final Segment) and Out-Link (Initial Segment)


## Single Attribute

###  Node Link-IN validation: FUNCTIONAL CLASS

In [14]:
### SPECIFY PARAMETERS
# IN Attribute:
attribute_name = ['FUNCCLASS']
# Filter by OUT attribute value (Equal to):
out_attribute_filters = [1,11,12]
#Export to csv name:
csv_name = 'Network_attribute_change_table'


### RUN FUNCTIONS
network = link_matcher_atts(attribute_name, graph=g)

# network[(network['in_FUNCCLASS']==98) & (network['out_FUNCCLASS'].isin([1,11,12]))]
network = summary_in(network, *attribute_name, csv_name, out_attribute_filters)
network.astype(int)

,in_FUNCCLASS,"out[1, 11, 12]"
0,1,101
1,2,6
2,7,12
3,11,742
4,12,469
5,14,101
6,16,101
7,17,16
8,18,6
9,19,17


###  Node Link-IN validation: NUMBER OF LANES

In [15]:
### SPECIFY PARAMETERS
# IN Attribute:
attribute_name = ['_LANES']
# Filter by OUT attribute value (Equal to):
out_attribute_filters = [5]
#Export to csv name:
csv_name = 'Network_attribute_change_table'


### RUN FUNCTIONS
network = link_matcher_atts(attribute_name, graph=g)

# network[(network['in_{}'.format(*attribute_name)]==1) & (network['out_{}'.format(*attribute_name)].isin([5]))].astype(int)
network = summary_in(network, *attribute_name, csv_name, out_attribute_filters)
network.astype(int)

,in__LANES,out[5]
0,1,5
1,2,2
2,3,1
3,4,2
4,5,12


###  Node Link-IN  Link-OUT Summary validation: SPEED LIMIT

In [16]:
### SPECIFY PARAMETERS
# IN Attribute:
attribute_name = ['SPD_LMT']
# Filter by OUT attribute value (Equal to):
in_attribute_filters = ''
out_attribute_filters = [70]
# Return Summary table or Link level table
return_summary = True
# Include Links where Attribute remains the same In/Out
include_same_att = False
#Export to csv name:
csv_name = 'Network_attribute_change_table'


### RUN FUNCTIONS
network = link_matcher_atts(attribute_name, graph=g)

# network[(network['in_{}'.format(*attribute_name)]==45) & (network['out_{}'.format(*attribute_name)].isin([70]))].astype(int)
network = summary_in_out(network, 
                         *attribute_name,
                         csv_name,
                         in_attribute_filters, 
                         out_attribute_filters, 
                         return_summary, 
                         include_same_att,
                        )
network.astype(int)

,in_SPD_LMT,out_SPD_LMT,count
0,45,70,7
1,65,70,2


###  Node Link-IN  Link-OUT Summary validation: FUNCTIONAL CLASS

In [17]:
### SPECIFY PARAMETERS
# IN Attribute:
attribute_name = ['FUNCCLASS']
# Filter by IN / OUT attribute value (Equal to):
in_attribute_filters = ''
out_attribute_filters = [11]
# Return Summary table or Link level table
return_summary = True
# Include Links where Attribute remains the same In/Out
include_same_att = False
#Export to csv name:
csv_name = 'Network_attribute_change_table'



### RUN FUNCTIONS
network = link_matcher_atts(attribute_name, graph=g)

#SUMMARY
network = summary_in_out(network, 
                         *attribute_name,
                         csv_name,
                         in_attribute_filters, 
                         out_attribute_filters, 
                         return_summary, 
                         include_same_att,
                        )
network.astype(int)

,in_FUNCCLASS,out_FUNCCLASS,count
0,1,11,4
1,2,11,1
2,12,11,5
3,14,11,51
4,16,11,55
5,17,11,9
6,19,11,11
7,98,11,2
8,99,11,3


###  Node Link-IN  Link-OUT Summary validation: RIGHT SHOULDER WIDTH

In [18]:
### SPECIFY PARAMETERS
# IN Attribute:
attribute_name = ['RS_WIDTH']
# Filter by IN / OUT attribute value (Equal to):
in_attribute_filters = ''
out_attribute_filters = [24, 30]
# Return Summary table or Link level table
return_summary = True
# Include Links where Attribute remains the same In/Out
include_same_att = False
#Export to csv name:
csv_name = 'Network_attribute_change_table'


### RUN FUNCTIONS
network = link_matcher_atts(attribute_name, graph=g)

# SUMMARY
network = summary_in_out(network, 
                         *attribute_name,
                         csv_name,
                         in_attribute_filters, 
                         out_attribute_filters, 
                         return_summary, 
                         include_same_att,
                        )
network

,in_RS_WIDTH,out_RS_WIDTH,count
0,10.0,24.0,1
1,10.0,30.0,3
2,12.0,24.0,1
3,20.0,24.0,1


###  Node Link-IN  Link-OUT Summary validation: (EXCESSIVE) SPEED LIMITS

In [19]:
### SPECIFY PARAMETERS
# IN Attribute:
attribute_name = ['SPD_LMT']
# Filter by IN / OUT attribute value (Equal to):
in_attribute_filters = ''
out_attribute_filters = [65, 70]
# Return Summary table or Link level table
return_summary = True
# Include Links where Attribute remains the same In/Out
include_same_att = False
#Export to csv name:
csv_name = 'Network_attribute_change_table'


### RUN FUNCTIONS
network = link_matcher_atts(attribute_name, graph=g)

# SUMMARY
network = summary_in_out(network, 
                         *attribute_name,
                         csv_name,
                         in_attribute_filters, 
                         out_attribute_filters, 
                         return_summary, 
                         include_same_att,
                        )
network

,in_SPD_LMT,out_SPD_LMT,count
0,35.0,65.0,4
1,45.0,65.0,45
2,45.0,70.0,7
3,55.0,65.0,4
4,60.0,65.0,2
5,65.0,70.0,2
6,70.0,65.0,4


## Multiple Attribute

### Node Link-IN  Link-OUT validation: SPEED LIMIT, and NUMBER OF LANES

In [20]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', '_LANES']
# Filter by CHANGE in attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':30,
#     '_LANES':1
}
# Summarize by Node Maximum value and Export Geo:
export_geometry = True


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, export_geometry)

### ADDITIONAL FILTERING
network[network['node'].isin([2137, 2663])].drop(columns=['geometry'])

Total network links with change in SPD_LMT, _LANES: 14
Table exported to ..\data\Network_attribute_change_table_SPD_LMT__LANES.csv
Shapefile exported to ..\data\Network_attribute_change_SPD_LMT.shp


,ID,node,in_ID,in_SPD_LMT,out_SPD_LMT,in__LANES,out__LANES,diff_SPD_LMT,diff__LANES
2,92,2137,10679,25.0,55.0,1.0,2.0,30.0,1.0
3,92,2137,10631,25.0,55.0,1.0,2.0,30.0,1.0
4,10631,2137,272,55.0,25.0,2.0,1.0,-30.0,-1.0
11,774,2663,10631,25.0,55.0,1.0,2.0,30.0,1.0
12,10679,2137,272,55.0,25.0,2.0,1.0,-30.0,-1.0
13,10631,2663,627,55.0,25.0,2.0,1.0,-30.0,-1.0


### Node Link-IN  Link-OUT validation: SPEED LIMIT, NUMBER OF LANES and HEADING

In [21]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', '_LANES', '_HEADING']
# Filter by CHANGE in attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':25,
#     '_LANES':1,
#     '_HEADING':30
}
# Summarize by Node Maximum value and Export Geo:
export_geometry = True


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, export_geometry)
network.drop(columns=['geometry'])

# ### ADDITIONAL FILTERING
network[abs(network['diff__HEADING'])<=30].drop(columns=['geometry'])

Total network links with change in SPD_LMT, _LANES, _HEADING: 46
Table exported to ..\data\Network_attribute_change_table_SPD_LMT__LANES__HEADING.csv
Shapefile exported to ..\data\Network_attribute_change_SPD_LMT.shp


,ID,node,in_ID,in_SPD_LMT,out_SPD_LMT,in__LANES,out__LANES,in__HEADING,out__HEADING,diff_SPD_LMT,diff__LANES,diff__HEADING
0,17366,16469,38569,45.0,70.0,2.0,2.0,58.921408,46.640245,25.0,0.0,-12.281162
1,17366,16469,17358,45.0,70.0,1.0,2.0,24.889821,46.640245,25.0,1.0,21.750424
5,33530,29066,33518,70.0,45.0,3.0,1.0,301.026323,306.857526,-25.0,-2.0,5.831203
6,33600,29065,33531,45.0,70.0,1.0,3.0,289.653367,298.987783,25.0,2.0,9.334416
8,17481,16608,38599,45.0,70.0,3.0,3.0,322.237216,315.104885,25.0,0.0,-7.132330
14,12686,12951,15443,55.0,30.0,1.0,1.0,323.943239,327.116635,-25.0,0.0,3.173396
15,38568,16801,17648,70.0,45.0,2.0,2.0,222.502069,236.525028,-25.0,0.0,14.022959
17,38598,16903,17769,70.0,45.0,3.0,3.0,136.384845,141.761863,-25.0,0.0,5.377019
27,33621,29160,33640,45.0,70.0,1.0,3.0,111.481814,119.091001,25.0,2.0,7.609187
28,33636,29159,33700,70.0,45.0,3.0,1.0,120.712426,129.499301,-25.0,-2.0,8.786875


### Network Link-to-Link attribute side-by-side validation: NUMBER OF LANES and FUNCTION CLASS

In [22]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['_LANES', 'FUNCCLASS', 'FUNC_CLASS']
# Filter by CHANGE in attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    '_LANES':3,
}
# Summarize by Node Maximum value and Export Geo:
export_geometry = True


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, export_geometry)
network.head()


### ADDITIONAL FILTERING
# network[network['node'].isin([15248])].drop(columns=['geometry', 'diff_FUNCCLASS'])

### ADDITIONAL FILTERING: None is IN/OUT FREEWAY
network[(~network['in_FUNCCLASS'].isin([1,11,12])) & (~network['out_FUNCCLASS'].isin([1,11,12]))]\
       .drop(columns=['geometry', 'diff_FUNCCLASS'])

Total network links with change in _LANES, FUNCCLASS, FUNC_CLASS: 47
Table exported to ..\data\Network_attribute_change_table__LANES_FUNCCLASS_FUNC_CLASS.csv
Shapefile exported to ..\data\Network_attribute_change__LANES.shp


,ID,node,in_ID,in__LANES,out__LANES,in_FUNCCLASS,out_FUNCCLASS,in_FUNC_CLASS,out_FUNC_CLASS,diff__LANES
8,2301,3181,1203,4.0,1.0,14.0,16.0,Urban Principal Arterial,Urban Minor Arterial,-3.0
9,1206,2583,78758,1.0,4.0,19.0,14.0,None,Urban Principal Arterial,3.0
29,2383,3875,6527,1.0,4.0,17.0,14.0,Urban Major Collector,Urban Principal Arterial,3.0
30,6527,3875,1843,4.0,1.0,14.0,17.0,Urban Principal Arterial,Urban Major Collector,-3.0
31,13542,3181,1203,4.0,1.0,14.0,16.0,Urban Principal Arterial,Urban Minor Arterial,-3.0


### Network Link-to-Link attribute side-by-side validation: SPEED LIMIT and ROUTE NAME
Filter by Speed Limit

In [23]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', 'RTE_NME']
# Filter by CHANGE in attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':10,
}
# Summarize by Node Maximum value and Export Geo:
export_geometry = False


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, export_geometry)
network.head()

Total network links with change in SPD_LMT, RTE_NME: 7,791
Table exported to ..\data\Network_attribute_change_table_SPD_LMT_RTE_NME.csv


,node,in_ID,out_ID,in_SPD_LMT,out_SPD_LMT,in_RTE_NME,out_RTE_NME,diff_SPD_LMT,diff_RTE_NME
14,11161,38587,10186,45.0,55.0,CENTROID CONNECTOR,GEORGETOWN RD NW,10.0,CHANGES
15,11161,10186,38587,55.0,45.0,GEORGETOWN RD NW,CENTROID CONNECTOR,-10.0,CHANGES
18,2158,284,110,45.0,65.0,None,US-27,20.0,CHANGES
22,2674,635,782,65.0,45.0,US-27,None,-20.0,CHANGES
158,2523,2308,487,45.0,55.0,TN-58,TN-58,10.0,CHANGES


### Network Link-to-Link attribute side-by-side validation: SPEED LIMIT and ROUTE NAME
Filter by Route Name

In [24]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', 'RTE_NME']
# Filter by CHANGE in attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':10,
    'RTE_NME':'CENTROID CONNECTOR',
}
# Summarize by Node Maximum value and Export Geo:
export_geometry = False


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, export_geometry)
network.head()

Total network links with change in SPD_LMT, RTE_NME: 5,522
Table exported to ..\data\Network_attribute_change_table_SPD_LMT_RTE_NME.csv


,node,in_ID,out_ID,in_SPD_LMT,out_SPD_LMT,in_RTE_NME,out_RTE_NME,diff_SPD_LMT,diff_RTE_NME
14,11161,38587,10186,45.0,55.0,CENTROID CONNECTOR,GEORGETOWN RD NW,10.0,CHANGES
15,11161,10186,38587,55.0,45.0,GEORGETOWN RD NW,CENTROID CONNECTOR,-10.0,CHANGES
486,7410,5814,37902,35.0,45.0,DELASHMITT RD,CENTROID CONNECTOR,10.0,CHANGES
487,7410,37902,5779,45.0,35.0,CENTROID CONNECTOR,DELASHMITT RD,-10.0,CHANGES
489,7410,37902,5814,45.0,35.0,CENTROID CONNECTOR,DELASHMITT RD,-10.0,CHANGES


### Network Link-to-Link attribute side-by-side validation: NUMBER OF LANES and FUNCTIONAL CLASS
Filter by Number of Lanes

In [25]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['_LANES', 'FUNCCLASS']
# Filter by CHANGE in attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    '_LANES':2,
}
# Summarize by Node Maximum value and Export Geo:
export_geometry = True


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, export_geometry)
network.head()

Total network links with change in _LANES, FUNCCLASS: 776
Table exported to ..\data\Network_attribute_change_table__LANES_FUNCCLASS.csv
Shapefile exported to ..\data\Network_attribute_change__LANES.shp


,ID,geometry,node,in_ID,in__LANES,out__LANES,in_FUNCCLASS,out_FUNCCLASS,diff__LANES,diff_FUNCCLASS
0,1690,"LINESTRING (2195168.847 289805.215, 2195117.25...",3734,11674,1.0,3.0,19.0,14.0,2.0,-5.0
1,11674,"LINESTRING (2195168.847 289805.215, 2195385.42...",3734,1689,3.0,1.0,14.0,19.0,-2.0,5.0
2,11674,"LINESTRING (2195168.847 289805.215, 2195385.42...",3734,1690,3.0,1.0,14.0,19.0,-2.0,5.0
3,153,"LINESTRING (2209220.826 268047.894, 2209150.14...",2079,55,1.0,3.0,12.0,12.0,2.0,0.0
4,1253,"LINESTRING (2208264.195 274497.534, 2208404.72...",3242,6504,1.0,3.0,19.0,14.0,2.0,-5.0
